In [1]:
!python SCAR_GNN.py

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ruixiang/PycharmProjects/SCAR/src/SCAR_GNN.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  nodeset["label"] = nodeset["node"].str.contains(r"(sbox|mixcolumn)",case=False, na=False).astype(bool) \
/Users/ruixiang/PycharmProjects/SCAR/src/SCAR_GNN.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | nodeset["Node"].str.contains(r"(sbox|mixcolumn)",case=False, na=False).astype(bool)
feature names: {'Hamming distance', 'Paths', 'xor', 'and', 'mux', 'Degree', 'or'}
num_features: 7, num_classes: 2
feature_names: {'Hamming 

In [2]:
!python Feature_Extract.py AES_PPRM1 Kin AES_PPRM1
!python Feature_Extract.py AES_PPRM3 Kin AES_PPRM3
!python Feature_Extract.py AES_TBL Kin AES_TBL
!python Feature_Extract.py RSA Kin RSA
!python Feature_Extract.py SABER pol_64bit_in SABER

Generating LALR tables
[INFO] Features written to ../test/AES_PPRM1_features.csv
[INFO] Edges written to ../test/AES_PPRM1_edges.csv
Generating LALR tables
[INFO] Features written to ../test/AES_PPRM3_features.csv
[INFO] Edges written to ../test/AES_PPRM3_edges.csv
Generating LALR tables
[INFO] Features written to ../test/AES_TBL_features.csv
[INFO] Edges written to ../test/AES_TBL_edges.csv
Generating LALR tables
[INFO] Features written to ../test/RSA_features.csv
[INFO] Edges written to ../test/RSA_edges.csv
Generating LALR tables
[INFO] Features written to ../test/SABER_features.csv
[INFO] Edges written to ../test/SABER_edges.csv


In [3]:
import os, glob
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from GNN import *
from GraphInformation import *

TEST_DIR = "../test"

feature_files = sorted(glob.glob(os.path.join(TEST_DIR, "*_features.csv")))
paired = []
for fpath in feature_files:
    base = os.path.basename(fpath).replace("_features.csv", "")
    epath = os.path.join(TEST_DIR, f"{base}_edges.csv")
    if os.path.exists(epath):
        paired.append((base, fpath, epath))

if not paired:
    print("No dataset pairs found under ./test (expect *_features.csv + *_edges.csv)")
else:
    print(f"Found {len(paired)} dataset(s):", [b for b,_,_ in paired])

leaky_module = {
    "AES_PPRM1": ["SBOX", "Mixcolumns", "MX"],
    "AES_PPRM3": ["Sbox", "Mixcolumns", "MX"],
    "AES_TBL": ["SBOX", "Mixcolumns", "MX"],
    "RSA": ["MODEXP_SEQ", "MULT_BLK"],
    "SABER": ["PMULTs"]
}

results = []

for base, ffeat, fedge in paired:
    test_nodeset = pd.read_csv(ffeat)
    test_edge = pd.read_csv(fedge)
    keywords = leaky_module.get(base, [])
    def contains_any(value, keywords):
        return any(kw in str(value) for kw in keywords)
    test_nodeset["label"] = test_nodeset["node"].apply(
        lambda x: 1 if contains_any(x, keywords) else 0
    )
    test_nodeset.to_csv(ffeat, index=False)

    (X, edges, edges_weights), feature_names, num_features, num_classes = graph_information(ffeat, fedge, "test")
    Y = test_nodeset["label"]

    model = GNNNodeClassifier(
        graph_info=(X, edges, edges_weights),
        num_classes=num_classes,
        hidden_units=hidden_units,
        dropout_rate=dropout_rate,
        name="gnn_model",
    )

    haha = model(tf.convert_to_tensor([0], dtype=tf.int32))
    model.load_weights("../out/gnn_weights.weights.h5")

    all_idx = tf.range(X.shape[0], dtype=tf.int32)
    probs = model(all_idx).numpy()

    if probs.shape[1] == 2:
        y_pred = probs.argmax(axis=1)
        score_for_pos = probs[:, 1]
    else:
        y_pred = (probs.squeeze(-1) >= 0.5).astype(np.int32)
        score_for_pos = probs.squeeze(-1)

    y_true = Y.argmax(1) if Y.ndim == 2 else Y

    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred)
    try:
        auc = roc_auc_score(y_true, score_for_pos)
    except Exception:
        auc = float('nan')

    results.append((base, acc, f1, auc))

    test_nodeset["prediction"] = y_pred
    test_nodeset.to_csv(ffeat + "_pred.csv", index=False)

for (base, acc, f1, auc) in results:
    print(f"for {base}: Acc={acc:.4f}  F1={f1:.4f}  AUC={auc:.4f}")

Found 5 dataset(s): ['AES_PPRM1', 'AES_PPRM3', 'AES_TBL', 'RSA', 'SABER']
feature names: ['Degree', 'Hamming distance', 'Paths', 'and', 'mux', 'or', 'xor']
tf.Tensor(
[[0.         0.         0.         0.         0.         0.
  0.2746643  0.32224554 0.         0.         0.2685398  0.
  0.         0.         0.         0.6475763  0.         0.3559035
  0.         0.12767556 0.         0.         0.         0.
  0.         0.         0.2603831  0.24059333 0.         0.16081959
  0.         0.        ]], shape=(1, 32), dtype=float32)
tf.Tensor(
[[4.6859294e-01 0.0000000e+00 1.5298532e-01 ... 0.0000000e+00
  1.1573208e-01 2.1031110e-01]
 [2.2298322e+02 1.0580376e+01 1.3461108e+02 ... 0.0000000e+00
  1.6975815e+01 1.2183729e+02]
 [5.0967270e-01 0.0000000e+00 1.7077710e-01 ... 6.6068187e-02
  1.4288974e-01 3.2369253e-01]
 ...
 [4.6156239e+01 1.9137837e+00 2.7402466e+01 ... 0.0000000e+00
  3.5584078e+00 2.5389801e+01]
 [1.4849547e+01 4.8619974e-01 8.5215302e+00 ... 2.1313636e-01
  1.1980608